In [1]:
import sys
IN_COLAB = 'google.colab' in sys.modules
USE_GITHUB = False
USE_DRIVE = False
USE_LOCAL = ~USE_DRIVE and ~USE_DRIVE

if IN_COLAB:
  !pip install wandb -qU
  from google.colab import runtime
  if USE_GITHUB:
    !git clone https://github.com/kejeon/in_dev_RN20Q.git
    %cd '/content/in_dev_RN20Q'
  elif USE_DRIVE:
    from google.colab import drive
    drive.mount('/content/drive')
    %cd '/content/drive/MyDrive/GitHub/in_dev_RN20Q'

import wandb
import torch
from model.resnet4c10q import ResNet20_Q
from model.resnet4c10 import resnet20
from mylib.KDResNetTrainer import ResNetTrainer

wandb.login(key='e0c11d3ff2bee4c8775ba05863038fdac671c043')

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


wandb: Currently logged in as: jke1994. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /Users/kejeon/.netrc


True

In [3]:
api = wandb.Api()
artifact = api.artifact('jke1994/ResNet20/model:v179')
artifact.download(root='./pretrained_model')

state_dict = torch.load('./pretrained_model/ckpt.pth', map_location=torch.device('cpu'))

# update all keys in state_dict to remove module.
state_dict['net'] = {k.replace('module.', ''): v for k, v in state_dict['net'].items()}

teacher_model = resnet20()
# load pretrained model for resnet20
teacher_model.load_state_dict(state_dict['net'])

wandb:   1 of 1 files downloaded.  


'./pretrained_model'

In [ ]:
lr = 0.01
arch_tag = "ResNet20Q"
batch_size = 128
dataset = "CIFAR10"

student_model = ResNet20_Q(a_bit=4, w_bit=1)

run = wandb.init(
    # set the wandb project where this run will be logged
    project="ResNet20_Q",

    # track hyperparameters and run metadata
    config={
    "learning_rate": lr,
    "architecture": arch_tag,
    "batch_size": batch_size,
    "dataset": dataset
    }
)

my_trainer = ResNetTrainer(dataset=dataset,
                           arch_tag=arch_tag,
                           student_model=student_model,
                           teacher_model=teacher_model,
                           device = 'cuda',
                           batch_size=batch_size)

my_trainer.train_script(200)

wandb.finish()

runtime.unassign()